In [14]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt


In [15]:
#discriminator model
discriminator = tf.keras.Sequential(
    [
        tf.keras.Input(shape= (28,28,1)),
        tf.keras.layers.Conv2D(6,5,padding='same'),
        tf.keras.layers.ReLU(),
        tf.keras.layers.MaxPool2D(pool_size= (2,2),strides= (2,2)),
        tf.keras.layers.Conv2D(16,5,padding='valid'),
        tf.keras.layers.ReLU(),
        tf.keras.layers.MaxPool2D(pool_size= (2,2), strides=(2,2)),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(1, activation= 'sigmoid')
    ]
)

discriminator.summary()

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_10 (Conv2D)           (None, 28, 28, 6)         156       
_________________________________________________________________
re_lu_7 (ReLU)               (None, 28, 28, 6)         0         
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 14, 14, 6)         0         
_________________________________________________________________
conv2d_11 (Conv2D)           (None, 10, 10, 16)        2416      
_________________________________________________________________
re_lu_8 (ReLU)               (None, 10, 10, 16)        0         
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 5, 5, 16)          0         
_________________________________________________________________
flatten_2 (Flatten)          (None, 400)              

In [16]:
# Generator model
noise_dim = 150
generator = tf.keras.Sequential(
    [
        tf.keras.Input(shape=(noise_dim,)),
        tf.keras.layers.Dense(5*5*16, activation = 'relu'),
        tf.keras.layers.Reshape((5,5,16)),
        tf.keras.layers.UpSampling2D(size = (2,2)),
        tf.keras.layers.Conv2D(16,5, padding = 'same',activation = 'relu'),
        tf.keras.layers.Conv2DTranspose(6,5,activation = 'relu'),
        tf.keras.layers.UpSampling2D(size = (2,2)),
        tf.keras.layers.Conv2D(1,5,padding= 'same',),
        tf.keras.layers.ReLU(max_value = 255)
    ]
)
generator.summary()

Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_6 (Dense)              (None, 400)               60400     
_________________________________________________________________
reshape_3 (Reshape)          (None, 5, 5, 16)          0         
_________________________________________________________________
up_sampling2d_6 (UpSampling2 (None, 10, 10, 16)        0         
_________________________________________________________________
conv2d_12 (Conv2D)           (None, 10, 10, 16)        6416      
_________________________________________________________________
conv2d_transpose_3 (Conv2DTr (None, 14, 14, 6)         2406      
_________________________________________________________________
up_sampling2d_7 (UpSampling2 (None, 28, 28, 6)         0         
_________________________________________________________________
conv2d_13 (Conv2D)           (None, 28, 28, 1)        

In [17]:
dataset = tf.keras.preprocessing.image_dataset_from_directory(
    directory= 'digit_data',
    label_mode = None,
    batch_size = 60,
    shuffle = True,
    image_size = (28,28),
    color_mode='grayscale'
)
dataset = dataset.map(lambda x : x/255)

Found 42000 files belonging to 1 classes.


In [18]:
# optimizers for the generator and discriminator

opt_gen = tf.keras.optimizers.Adam(lr = 0.0001)
opt_disc = tf.keras.optimizers.Adam(lr = 0.0001)

# loss funtion

loss = tf.keras.losses.BinaryCrossentropy()

# the complete dcgan model has only one loss to which generator tries to maximize and discriminator tries to minimize 

In [19]:
for epoch in range(25):
    for idx, real in enumerate(dataset):
        batch_size = real.shape[0]
        random_latent_space = tf.random.normal(shape = (batch_size, noise_dim))

        fake = generator(random_latent_space)

        if idx % 50 == 0:
            img = tf.keras.preprocessing.image.array_to_img(fake[0])
            img.save(f'generated_images/gen{epoch}_{idx}.jpg')

        #training the discriminator 
        with tf.GradientTape() as disc_tape:
            loss_real = loss(tf.ones((batch_size,1)),discriminator(real))
            loss_fake = loss(tf.zeros((batch_size,1)),discriminator(fake))
            loss_disc = tf.add(loss_real,loss_fake)

        grads = disc_tape.gradient(loss_disc,discriminator.trainable_weights) 
        opt_disc.apply_gradients(zip(grads, discriminator.trainable_weights))

        # training the generator
        #for epoch_gen in range(2):
        with tf.GradientTape() as gen_tape:
            random_latent_space = tf.random.normal(shape = (batch_size, noise_dim))
            fake = generator(random_latent_space)
            loss_gen = loss(tf.ones((batch_size,1)),discriminator(fake))

        grads = gen_tape.gradient(loss_gen, generator.trainable_weights)
        opt_gen.apply_gradients(zip(grads,generator.trainable_weights))


